# Geolocalización Colombia

#### Librerias

In [7]:
import pandas as pd
import requests
import unicodedata
import os
import numpy as np

#### Conexiones

In [1]:
url_api = 'https://api.openweathermap.org/data/2.5/weather?'
api_key = 'd3c9b45e4711eb532f82fab35fc5fb53'
header = { "Content-Type":"aplication/json",
            "Accept-Encoding": "deflate"}

#### Funciones 

In [3]:
def datos_geo(Pais,Zip,Ciudad,url_api,api_key):
    end_p = f"{url_api}q={Ciudad},{Zip},{Pais}&appid={api_key}&units=metric"
    res = requests.get(end_p,headers=header)

    if res.status_code == 200:
        return res.json()
    
    else:
        None


def quitar_tildes(texto):
    if isinstance(texto, str):  
        texto = unicodedata.normalize('NFKD', texto).encode('ascii', 'ignore').decode('utf-8')
        return texto
    return texto


def coordenadas(df_mun, df_Geo):

    for i in range(len(df_mun)):
       
        if pd.isnull(df_mun.at[i, 'Latitud']) or pd.isnull(df_mun.at[i, 'Longitud']):
            
            df_mun.at[i, 'Latitud'] = df_Geo.at[i, 'Latitud']
            df_mun.at[i, 'Longitud'] = df_Geo.at[i, 'Longitud']
        else:
            
            if (df_mun.at[i, 'Latitud'] != df_Geo.at[i, 'Latitud'] or
                df_mun.at[i, 'Longitud'] != df_Geo.at[i, 'Longitud']):
                
                df_mun.at[i, 'Latitud'] = df_Geo.at[i, 'Latitud']
                df_mun.at[i, 'Longitud'] = df_Geo.at[i, 'Longitud']

#### Code

Carga de Data Frames

In [5]:
# df_mun = pd.read_excel('W:/Datasets/Municipios.xlsx')
# df_mun.sort_values(by=['Nombre_depto', 'Nombre_mpio'], ascending=[True, True], inplace=True)

# df_mun['Latitud'] = None
# df_mun['Longitud'] = None

# df_Geo = pd.read_excel('W:/Datasets/Geo_Municipios.xlsx')
# df_Geo.sort_values(by=['Nombre_depto', 'Nombre_mpio'], ascending=[True, True], inplace=True)


df_mun = pd.read_excel(r'C:\Users\Darryl\Desktop\Proyecto\Municipios.xlsx')
df_mun.sort_values(by=['Nombre_depto', 'Nombre_mpio'], ascending=[True, True], inplace=True)

df_mun['Latitud'] = None
df_mun['Longitud'] = None

df_Geo = pd.read_excel(r'C:\Users\Darryl\Desktop\Proyecto\Geo_Municipios.xlsx')
df_Geo.sort_values(by=['Nombre_depto', 'Nombre_mpio'], ascending=[True, True], inplace=True)

Agregar latitud y longitud al data frame municipios a traves de la api y normalizar el df

In [8]:
for i, row in df_mun.iterrows():

    mpio = row['Nombre_mpio']
    cod_mpio = row['Cod_mpio'] 

    datos = datos_geo('CO', cod_mpio, mpio, url_api, api_key)
    
    if datos:
        df_mun.at[i, 'Latitud'] = np.float64(datos['coord']['lat'])
        df_mun.at[i, 'Longitud'] = np.float64(datos['coord']['lon'])

coordenadas(df_mun, df_Geo)

df_mun = df_mun.apply(lambda col: col.map(lambda x: quitar_tildes(x.lower()) if isinstance(x, str) else x))


KeyboardInterrupt: 